Before you submit this notebook, make sure everything runs as expected in the local test cases. 
Please, paste the solution to the designed cell and do not change anything else.

Also, please, leave your first and last names below

In [ ]:
FirstName = "Musa"
LastName = "Safin"

---

In [ ]:
import numpy as np

In [ ]:
class Module(object):
    """
    Basically, you can think of a module as of a something (black box) 
    which can process `input` data and produce `ouput` data.
    This is like applying a function which is called `forward`: 
        
        output = module.forward(input)
    
    The module should be able to perform a backward pass: to differentiate the `forward` function. 
    More, it should be able to differentiate it if is a part of chain (chain rule).
    The latter implies there is a gradient from previous step of a chain rule. 
    
        gradInput = module.backward(input, gradOutput)
    """
    def __init__ (self):
        self.output = None
        self.gradInput = None
        self.training = True
    
    def forward(self, input):
        """
        Takes an input object, and computes the corresponding output of the module.
        """
        return self.updateOutput(input)

    def backward(self,input, gradOutput):
        """
        Performs a backpropagation step through the module, with respect to the given input.
        
        This includes 
         - computing a gradient w.r.t. `input` (is needed for further backprop),
         - computing a gradient w.r.t. parameters (to update parameters while optimizing).
        """
        self.updateGradInput(input, gradOutput)
        self.accGradParameters(input, gradOutput)
        return self.gradInput
    

    def updateOutput(self, input):
        """
        Computes the output using the current parameter set of the class and input.
        This function returns the result which is stored in the `output` field.
        
        Make sure to both store the data in `output` field and return it. 
        """
        
        # The easiest case:
            
        # self.output = input 
        # return self.output
        
        pass

    def updateGradInput(self, input, gradOutput):
        """
        Computing the gradient of the module with respect to its own input. 
        This is returned in `gradInput`. Also, the `gradInput` state variable is updated accordingly.
        
        The shape of `gradInput` is always the same as the shape of `input`.
        
        Make sure to both store the gradients in `gradInput` field and return it.
        """
        
        # The easiest case:
        
        # self.gradInput = gradOutput 
        # return self.gradInput
        
        pass   
    
    def accGradParameters(self, input, gradOutput):
        """
        Computing the gradient of the module with respect to its own parameters.
        No need to override if module has no parameters (e.g. ReLU).
        """
        pass
    
    def zeroGradParameters(self): 
        """
        Zeroes `gradParams` variable if the module has params.
        """
        pass
        
    def getParameters(self):
        """
        Returns a list with its parameters. 
        If the module does not have parameters return empty list. 
        """
        return []
        
    def getGradParameters(self):
        """
        Returns a list with gradients with respect to its parameters. 
        If the module does not have parameters return empty list. 
        """
        return []
    
    def train(self):
        """
        Sets training mode for the module.
        Training and testing behaviour differs for Dropout, BatchNorm.
        """
        self.training = True
    
    def evaluate(self):
        """
        Sets evaluation mode for the module.
        Training and testing behaviour differs for Dropout, BatchNorm.
        """
        self.training = False
    
    def __repr__(self):
        """
        Pretty printing. Should be overrided in every module if you want 
        to have readable description. 
        """
        return "Module"

In [ ]:
class Linear(Module):
    """
    A module which applies a linear transformation 
    A common name is fully-connected layer, InnerProductLayer in caffe. 

    The module should work with 2D input of shape (n_samples, n_feature).
    """
    def init(self, n_in, n_out):
        super(Linear, self).init()

        # This is a nice initialization
        stdv = 1./np.sqrt(n_in)
        self.W = np.random.uniform(-stdv, stdv, size = (n_in, n_out))
        self.b = np.random.uniform(-stdv, stdv, size = n_out)

        self.gradW = np.zeros_like(self.W)
        self.gradb = np.zeros_like(self.b)

    def updateOutput(self, input):
        self.output = np.matmul(input, self.W) + self.b
        ################################################
        # your code here 
        ################################################
        return self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput = np.matmul(gradOutput, self.W.T)
        ################################################
        # your code here 
        ################################################
        return self.gradInput

    def accGradParameters(self, input, gradOutput):
        # input(batch_size, n_in), gradOutput(batch_size, n_out), gradW(n_in, n_out)
        self.gradW += np.matmul(input.T, gradOutput) # == input.T @ gradOutput
 
        self.gradb += gradOutput.sum(axis=0) # gradOutput(batch_size, n_out), gradb(n_out)
        ################################################
        # your code here 
        ################################################

    def zeroGradParameters(self):
        self.gradW.fill(0)
        self.gradb.fill(0)

    def getParameters(self):
        return [self.W, self.b]

    def getGradParameters(self):
        return [self.gradW, self.gradb]

    def repr(self):
        s = self.W.shape
        q = 'Linear %d -> %d' %(s[1],s[0])
        return q

In [ ]:
class Sequential(Module):
    """
         This class implements a container, which processes `input` data sequentially. 
         
         `input` is processed by each module (layer) in self.modules consecutively.
         The resulting array is called `output`. 
    """
    
    def __init__ (self):
        super(Sequential, self).__init__()
        self.modules = []
   
    def add(self, module):
        """
        Adds a module to the container.
        """
        self.modules.append(module)
        
    def remove(self):
        """
        Delete last module from the container.
        """
        self.modules.pop()

    def updateOutput(self, input):
        """
        Basic workflow of FORWARD PASS:
        
            y_0    = module[0].forward(input)
            y_1    = module[1].forward(y_0)
            ...
            output = module[n-1].forward(y_{n-2})   
            
            
        Just write a little loop. 
        """
        #self.y_list=[i for i   in range(len(self.modules))] 
        self.output = None
        for ind, module in enumerate(self.modules):
#             if ind ==0:
#                 self.y_list[0]=module.forward(input)
#             else:
#                 self.y_list[ind]=module.forward(self.y_list[ind-1])
            if self.output is None:
                 self.output = module.forward(input)
            else:
                self.output = module.forward(self.output)
                #???
        #raise NotImplementedError
        return self.output

    def backward(self, input, gradOutput):
        """
        Workflow of BACKWARD PASS:
            
            g_{n-1} = module[n-1].backward(y_{n-2}, gradOutput)
            g_{n-2} = module[n-2].backward(y_{n-3}, g_{n-1})
            ...
            g_1 = module[1].backward(y_0, g_2)   
            gradInput = module[0].backward(input, g_1)   
             
             
        !!!
                
        To each module you need to provide the input, module saw while forward pass, 
        it is used while computing gradients. 
        Make sure that the input for `i-th` layer the output of `module[i]` (just the same input as in forward pass) 
        and NOT `input` to this Sequential module. 
        
        !!!
        
        """
        #self.output=[i for i   in range(len(self.modules))]
        n=len(self.modules)
        self.gradInput = None
        for n in reversed(range(1, len(self.modules))):
        #while n>0:
            #n-=1
#             if n ==len(self.modules)-1:
#                 self.g_list[n]=self.modules[n].backward(self.y_list[n-1], gradOutput)
#                 print('backward from the end')
#             else: 
#                 self.g_list[n]=self.modules[n].backward(self.y_list[n-1], self.g_list[n-1])
            if self.gradInput is None:
            #if n ==len(self.modules)-1:
                self.gradInput=self.modules[n].backward(self.modules[n-1].output, gradOutput)
                print('backward from the end')
            else: 
                self.gradInput=self.modules[n].backward(self.modules[n-1].output, self.gradInput)
        #raise NotImplementedError
        #self.gradInput =  self.modules[0].backward(input, self.g_list[0]) 
        self.gradInput =  self.modules[0].backward(input, self.gradInput) 
        return self.gradInput
      

    def zeroGradParameters(self): 
        for module in self.modules:
            module.zeroGradParameters()
    
    def getParameters(self):
        """
        Should gather all parameters in a list.
        """
        return [x.getParameters() for x in self.modules]
    
    def getGradParameters(self):
        """
        Should gather all gradients w.r.t parameters in a list.
        """
        return [x.getGradParameters() for x in self.modules]
    
    def __repr__(self):
        string = "".join([str(x) + '\n' for x in self.modules])
        return string
    
    def __getitem__(self,x):
        return self.modules.__getitem__(x)

In [ ]:
class Linear(Module):
    """
    A module which applies a linear transformation 
    A common name is fully-connected layer, Dense in Keras, Linear in PyTorch. 
    
    The module should work with 2D input of shape (n_samples, n_feature).
    """
    def __init__(self, n_in, n_out, learning_rate=0.1):
        super(Linear, self).__init__()
       
        # This is a nice initialization
        stdv = 1./np.sqrt(n_in)
        self.W = np.random.uniform(-stdv, stdv, size=(n_out, n_in))
        self.b = np.random.uniform(-stdv, stdv, size=n_out)
        
        self.gradW = np.zeros_like(self.W)
        self.gradb = np.zeros_like(self.b)
        self.learning_rate = learning_rate
        
    def updateOutput(self, input):
        """forward pass"""
        self.output=[]
        #print('start linear updateOutput')
        #print(len(input))
        #for ind, i in enumerate(input):
            #print(len(i))
            #per_list=[]
            #для регуляризации добавьте + reg*W к апдейту весов линейного слоя
#             for ind_per, _ in enumerate(self.W):
#                 per=0
#                 #self.output.append(input[ind])
#                 #per.append(input[ind])
#                 #for ind1, _ in enumerate(i):
#                     #print("X=", self.output[ind] )
#                     #print("X[ind1]=", self.output[ind][ind1] )
#                     #print(len(self.W))
#                     #print(len(self.W[0]))
#                     #print("W=", self.W)
#                     #print("W[ind][0]=", self.W[ind1][0])
#                     #self.output[ind][ind1]*=self.W[ind_per][ind1]
#                     per+=input[ind]*self.W[ind_per][ind]
#                 per+=self.b[ind_per]
#                 per_list.append(per)
#             self.output.append(per_list)
        self.output=np.dot(input, self.W.transpose())
        self.output=self.output+self.b
        #print('finish linear updateOutput')
        #print(len(self.output))
        #print(self.output)
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        #raise NotImplementedError
        print('back throw linear layer')
        #print('input = ', input)
        #print('input.shape: ', input.shape)
        #self.gradInput = np.dot(self.W.transpose(), gradOutput.transpose())
        #self.gradInput=self.gradInput.transpose()
        self.gradInput = np.dot(gradOutput, self.W)
        #print('gradInput: ', self.gradInput)
        #print('gradInput.shape: ', self.gradInput.shape)
        #print('gradInput.shape: ', self.gradInputt.shape)
        return self.gradInput
    
    def accGradParameters(self, input, gradOutput):
        #raise NotImplementedError
        #print(len(gradOutput))
#         for ind, i in enumerate(self.gradW):
#             self.gradW[ind] -= optimizer_config['learning_rate']*gradOutput
#         self.gradb -= optimizer_config['learning_rate']*gradOutput
        #print(type(input))
        #print('input:', input)
        #print(type(gradOutput))
        #print('gradOutput', gradOutput)
#         for i in range(self.W.shape[0]):
# #             self.gradW[i] *= 0
# #             self.gradb[i] = 0
#             self.zeroGradParameters()
#             for sample, grad in zip(input, gradOutput[i]):
#                 self.gradW[i] += sample * grad
#                 self.gradb[i] += grad
        for sample, grad in zip(input, gradOutput):
            grad1=grad.reshape(-1,1)
            #print('grad reshaped: ', grad1)
            sample1=sample.reshape(1, -1)
            #print('input reshaped: ', sample1)
            self.gradb+=grad
            gradW=grad1 @ sample1
            
#             self.gradW=np.hstack((self.gradW, gradW))
#             self.gradb=np.hstack((self.gradb, grad))
        #print('self.gradW', self.gradW)
        #print('self.gradb', self.gradb)
        
    def zeroGradParameters(self):
        self.gradW.fill(0)
        self.gradb.fill(0)
        
    def getParameters(self):
        return [self.W, self.b]
    
    def getGradParameters(self):
        return [self.gradW, self.gradb]
    
    def __repr__(self):
        s = self.W.shape
        q = 'Linear %d -> %d' %(s[1],s[0])
        return q

In [ ]:
class SoftMax(Module):
    def __init__(self):
         super(SoftMax, self).__init__()
    
    def updateOutput(self, input):
        # start with normalization for numerical stability
        self.output = np.subtract(input, input.max(axis=1, keepdims=True))
        
        # Your code goes here. ################################################
        
        np.exp(self.output, out=self.output)
        np.divide(self.output, np.sum(self.output, axis=1, keepdims=True), out=self.output)
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        
        # dz/dx_ij = sum_l(dz/dy_il * dy_il/dx_ij) = -sum_l(dz/dy_il * y_il * y_ij) + dz/dy_ij * y_ij =
        # ... = [-dz/dY_i * Y_i + dz/dy_ij] * y_ij
        self.gradInput = np.zeros_like(gradOutput)
        # we are gonna get (-dz/dY_i * Y_i) for each row and then add this column to dz/dy
        np.subtract(gradOutput, np.sum(np.multiply(gradOutput, self.output), axis=1, keepdims=True), out=self.gradInput)
        # now we have the [] from the equation above and need to just multiply it by Y
        np.multiply(self.gradInput, self.output, out=self.gradInput)
        return self.gradInput
    
    def __repr__(self):
        return "SoftMax"

In [ ]:
class LogSoftMax(Module):
    def __init__(self):
         super(LogSoftMax, self).__init__()
    
    def softmax(self, input):
        #raise NotImplementedError
        #print('input ', input)
        input_ = input - np.max(input)
        #print('input_ ', input_)
        sum_ = np.sum(np.exp(input_))
        return np.exp(input_)/sum_
    
    def updateOutput(self, input):
        # start with normalization for numerical stability
        self.output = np.subtract(input, np.max(input, axis=1, keepdims=True))
        #raise NotImplementedError
        exp=0
        print('Update LogSoftMax output')
        #print('input:', input)
#         for ind, _ in enumerate(input):
#             for  j in input[ind]:
#                 exp+=math.exp(j)
#             for ind_i, i in enumerate(self.output[ind]):
#                 self.output[ind][ind_i]=(i-math.log(exp))
        for i in range(self.output.shape[0]):
            self.output[i] = self.output[i] - np.log(np.sum(np.exp(self.output[i])))
        return self.output
        #print('logsoftmax=', self.output)
        #return self.output
    
    def updateGradInput(self, input, gradOutput):
        self.gradInput = np.zeros(input.shape)
        print('back throw logsoftmax')
        #print('input ', self.gradInput)
        #print('output', gradOutput)
        #for ind, _ in enumerate(input):
            #for ind_j, j in enumerate(input[ind]):
                #(softmax(X) - Y) / batch_size
#         for ind, i in enumerate(input):
#             exp=0
#             for  j in input[ind]:
#                 exp+=math.exp(j)
#             self.gradInput[ind] = (1 - (np.exp(input[ind]) / exp))*gradOutput[ind]
        #self.gradInput = np.zeros(input.shape)
        for i in range(self.gradInput.shape[0]):
            s = self.softmax(input[i])
            #print('s ', type(s) )
            self.gradInput[i] = s / input.shape[0] + gradOutput[i]
        return self.gradInput
        #print('input ', self.gradInput)
        #return self.gradInput
    
    def __repr__(self):
        return "LogSoftMax"

In [ ]:
class BatchNormalization(Module):
    EPS = 1e-3
    def __init__(self, alpha = 0.):
        super(BatchNormalization, self).__init__()
        self.alpha = alpha
        self.moving_mean = None 
        self.moving_variance = None
        
    def updateOutput(self, input):
        # Your code goes here. ################################################
        # use self.EPS please
        batch_size, n_features = input.shape
        self.output = np.zeros_like(input)
        
        if self.training == True:
            # Batch mean
            mean = np.mean(input, axis=0)
        
            # Batch variance
            variance = np.zeros_like(input)
            np.subtract(input, mean, out=variance)
            np.square(variance, out=variance)
            variance = np.mean(variance, axis=0)

            # Moving mean
            if self.moving_mean is None:
                self.moving_mean = np.zeros_like(mean)
            np.multiply(self.moving_mean, self.alpha, out=self.moving_mean)
            np.add(self.moving_mean, mean * (1 - self.alpha), out=self.moving_mean)

            # Moving variance
            if self.moving_variance is None:
                self.moving_variance = np.zeros_like(variance)
            np.multiply(self.moving_variance, self.alpha, out=self.moving_variance)
            np.add(self.moving_variance, variance * (1 - self.alpha), out=self.moving_variance)
        else:
            # During test or validation stage
            mean = self.moving_mean
            variance = self.moving_variance
        
        # Calculate x_hat using input, mean, and variance
        denominator = np.zeros_like(variance)
        np.add(variance, self.EPS, out=denominator)
        np.sqrt(denominator, out=denominator)
        np.subtract(input, mean, out=self.output) # numerator
        np.divide(self.output, denominator, out=self.output)
        
        # Store denominator for backprop
        self.denominator = denominator
        
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        batch_size, n_features = input.shape
        self.gradInput = np.zeros_like(gradOutput)
        
        # Derivation
        # For each x_i (input row) in the batch:
        # dz/dx_i = dz/dy_i * dy_i/dx_i + dz/dmean * dmean/dx_i + dz/dvariance * dvariance/dx_i
        ####
        # 1. dy_i/dx_i = 1 / denominator
        # 2. dz/dmean = sum_k(dz/dy_k * dy_k/dmean) + dz/dvariance * dvariance/dmean
        #      a) dy_i/dmean = -1 / denominator
        #      b) dvariance/dmean = -2 / N * sum_k(x_k - mean) = 0
        #    Putting a)-b) into 2. we get:
        #    dz/dmean = -1 / denominator * sum_k(dz/dy_k)
        # 3. dmean/dx_i = 1 / N
        # 4. dz/dvariance = sum_k(dz/dy_k * dy_k/dvariance) = -1 / (2 * denominator ** 2) * sum_k(dz/dy_k * y_k)
        # 5. dvariance/dx_i = 2 * (x_i - mean) / N
        ####
        # Putting 1.-5. into the equation above we finally get:
        # dz/x_i = 1 / denominator * [dz/dy_i - (sum_k(dz/dy_k) + y_i * sum_k(dz/dy_k * y_k)) / N]

        #### Here I use the denominator saved from the forward pass.
        denominator = self.denominator
        #### Alternatively, we can recalculate it here (commented code):
        
#         # Batch mean
#         mean = np.mean(input, axis=0, out=mean)
        
#         input_centered = np.subtract(input, mean)

#         # Batch variance
#         variance = np.zeros_like(input)
#         np.subtract(input, mean, out=variance)
#         np.square(variance, out=variance)
#         variance = np.mean(variance, axis=0)

#         # sqrt(variance + epsilon)
#         denominator = np.zeros_like(variance)
#         np.add(variance, self.EPS, out=denominator)
#         np.sqrt(denominator, out=denominator)
        
        np.multiply(gradOutput, self.output, out=self.gradInput)
        np.multiply(self.output, np.sum(self.gradInput, axis=0), out=self.gradInput)
        np.add(self.gradInput, np.sum(gradOutput, axis=0), out=self.gradInput)
        np.divide(self.gradInput, batch_size, out=self.gradInput)
        np.negative(self.gradInput, out=self.gradInput)
        np.add(self.gradInput, gradOutput, out=self.gradInput)
        np.divide(self.gradInput, denominator, out=self.gradInput)

        #### Alternatively, using chain rule as in the original paper:

#         dldvariance = np.multiply(-.5 * self.output / np.add(variance, self.EPS), gradOutput).sum(axis=0)
        
#         dldmean = -2 * input_centered.sum(axis=0) / batch_size
#         np.multiply(dldvariance, dldmean, out=dldmean)
#         np.add(np.divide(gradOutput, -denominator).sum(axis=0), dldmean, out=dldmean)
        
#         self.gradInput = np.zeros_like(input)
#         np.add(np.divide(gradOutput, denominator), self.gradInput, out=self.gradInput)
#         np.add(np.multiply(dldvariance, 2 * input_centered / batch_size), self.gradInput, out=self.gradInput)
#         np.add(np.multiply(dldmean, 1 / batch_size), self.gradInput, out=self.gradInput)

        return self.gradInput
    
    def __repr__(self):
        return "BatchNormalization"

In [ ]:
class ChannelwiseScaling(Module):
    """
       Implements linear transform of input y = \gamma * x + \beta
       where \gamma, \beta - learnable vectors of length x.shape[-1]
    """
    def __init__(self, n_out):
        super(ChannelwiseScaling, self).__init__()

        stdv = 1./np.sqrt(n_out)
        self.gamma = np.random.uniform(-stdv, stdv, size=n_out)
        self.beta = np.random.uniform(-stdv, stdv, size=n_out)
        
        self.gradGamma = np.zeros_like(self.gamma)
        self.gradBeta = np.zeros_like(self.beta)

    def updateOutput(self, input):
        self.output = input * self.gamma.astype(input.dtype) + self.beta.astype(input.dtype)
        return self.output
        
    def updateGradInput(self, input, gradOutput):
        self.gradInput = gradOutput * self.gamma.astype(input.dtype)
        return self.gradInput
    
    def accGradParameters(self, input, gradOutput):
        self.gradBeta = np.sum(gradOutput, axis=0)
        self.gradGamma = np.sum(gradOutput*input, axis=0)
        
    def zeroGradParameters(self):
        self.gradGamma.fill(0)
        self.gradBeta.fill(0)
        
    def getParameters(self):
        return [self.gamma, self.beta]
    
    def getGradParameters(self):
        return [self.gradGamma, self.gradBeta]
    
    def __repr__(self):
        return "ChannelwiseScaling"

In [ ]:
class Dropout(Module):
    def __init__(self, p=0.5):
        super(Dropout, self).__init__()
        
        self.p = p
        self.mask = None
        
    def updateOutput(self, input):
        # Your code goes here. ################################################
        self.output = input.copy()
        if self.training == True:
            q = 1. - self.p # prob of "1" in the bernoulli mask
            self.mask = np.random.binomial(1, q, size=input.shape)
            np.multiply(self.output, self.mask, out=self.output)
            np.divide(self.output, q, out=self.output)
        return  self.output
    
    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        self.gradInput = np.zeros_like(gradOutput)
        q = 1. - self.p
        np.multiply(gradOutput, self.mask, out=self.gradInput)
        np.divide(self.gradInput, q, out=self.gradInput)
        return self.gradInput
        
    def __repr__(self):
        return "Dropout"

In [ ]:
class ReLU(Module):
    def __init__(self):
         super(ReLU, self).__init__()
    
    def updateOutput(self, input):
        self.output = np.maximum(input, 0)
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        self.gradInput = np.multiply(gradOutput , input > 0)
        return self.gradInput
    
    def __repr__(self):
        return "ReLU"

In [ ]:
class LeakyReLU(Module):
    def __init__(self, slope = 0.03):
        super(LeakyReLU, self).__init__()
            
        self.slope = slope
        
    def updateOutput(self, input):
        # Your code goes here. ################################################
        self.output = np.maximum(input, self.slope * input)
        return  self.output
    
    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        self.gradInput = gradOutput.copy()
        self.gradInput[input < 0] *= self.slope
        return self.gradInput
    
    def __repr__(self):
        return "LeakyReLU"

In [ ]:
class ELU(Module):
    def __init__(self, alpha = 1.0):
        super(ELU, self).__init__()
        
        self.alpha = alpha
        
    def updateOutput(self, input):
        # Your code goes here. ################################################
        self.output = input.copy()
        self.output[input < 0] = self.alpha * (np.exp(input[input < 0]) - 1)
        return  self.output
    
    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        self.gradInput = self.output + self.alpha
        self.gradInput[input > 0] = 1
        self.gradInput *= gradOutput
        return self.gradInput
    
    def __repr__(self):
        return "ELU"

In [ ]:
class SoftPlus(Module):
    def __init__(self):
        super(SoftPlus, self).__init__()
    
    def updateOutput(self, input):
        # Your code goes here. ################################################
        self.output = np.exp(input)
        np.add(1., self.output, out=self.output)
        np.log(self.output, out=self.output)
        return  self.output
    
    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        # dy/dx = 1 - 1/(1 + exp(x))
        self.gradInput = np.exp(input)
        np.add(1., self.gradInput, out=self.gradInput)
        np.reciprocal(self.gradInput, out=self.gradInput)
        np.subtract(1., self.gradInput, out=self.gradInput)
        
        np.multiply(self.gradInput, gradOutput, out=self.gradInput)
        return self.gradInput
    
    def __repr__(self):
        return "SoftPlus"

In [ ]:
class Criterion(object):
    def __init__ (self):
        self.output = None
        self.gradInput = None
        
    def forward(self, input, target):
        """
            Given an input and a target, compute the loss function 
            associated to the criterion and return the result.
            
            For consistency this function should not be overrided,
            all the code goes in `updateOutput`.
        """
        return self.updateOutput(input, target)

    def backward(self, input, target):
        """
            Given an input and a target, compute the gradients of the loss function
            associated to the criterion and return the result. 

            For consistency this function should not be overrided,
            all the code goes in `updateGradInput`.
        """
        return self.updateGradInput(input, target)
    
    def updateOutput(self, input, target):
        """
        Function to override.
        """
        return self.output

    def updateGradInput(self, input, target):
        """
        Function to override.
        """
        return self.gradInput   

    def __repr__(self):
        """
        Pretty printing. Should be overrided in every module if you want 
        to have readable description. 
        """
        return "Criterion"

In [ ]:
class MSECriterion(Criterion):
    def __init__(self):
        super(MSECriterion, self).__init__()
        
    def updateOutput(self, input, target):   
        self.output = np.sum(np.power(input - target,2)) / input.shape[0]
        return self.output 
 
    def updateGradInput(self, input, target):
        self.gradInput  = (input - target) * 2 / input.shape[0]
        return self.gradInput

    def __repr__(self):
        return "MSECriterion"

In [ ]:
class ClassNLLCriterionUnstable(Criterion):
    EPS = 1e-15
    def __init__(self):
        a = super(ClassNLLCriterionUnstable, self)
        super(ClassNLLCriterionUnstable, self).__init__()
        
    def updateOutput(self, input, target): 
        
        # Use this trick to avoid numerical errors
        input_clamp = np.clip(input, self.EPS, 1 - self.EPS)
        
        # Your code goes here. ################################################        
        self.output = -np.sum(np.log(input_clamp[target == 1])) / input.shape[0]
        return self.output

    def updateGradInput(self, input, target):
        
        # Use this trick to avoid numerical errors
        input_clamp = np.clip(input, self.EPS, 1 - self.EPS)
                
        # Your code goes here. ################################################
        self.gradInput = np.divide(target, input_clamp)
        np.divide(self.gradInput, -input.shape[0], out=self.gradInput)
        return self.gradInput
    
    def __repr__(self):
        return "ClassNLLCriterionUnstable"

In [ ]:
class ClassNLLCriterion(Criterion):
    def __init__(self):
        a = super(ClassNLLCriterion, self)
        super(ClassNLLCriterion, self).__init__()
        
    def updateOutput(self, input, target): 
        # Your code goes here. ################################################
        self.output = -np.sum(input[target == 1]) / input.shape[0]
        return self.output

    def updateGradInput(self, input, target):
        # Your code goes here. ################################################
        self.gradInput = np.divide(target, -input.shape[0])
        return self.gradInput
    
    def __repr__(self):
        return "ClassNLLCriterion"

In [ ]:
def sgd_momentum(variables, gradients, config, state):  
    # 'variables' and 'gradients' have complex structure, accumulated_grads will be stored in a simpler one
    state.setdefault('accumulated_grads', {})
    
    var_index = 0 
    for current_layer_vars, current_layer_grads in zip(variables, gradients): 
        for current_var, current_grad in zip(current_layer_vars, current_layer_grads):
            
            old_grad = state['accumulated_grads'].setdefault(var_index, np.zeros_like(current_grad))
            
            np.add(config['momentum'] * old_grad, config['learning_rate'] * current_grad, out=old_grad)
            
            current_var -= old_grad
            var_index += 1   

In [ ]:
def adam_optimizer(variables, gradients, config, state):  
    # 'variables' and 'gradients' have complex structure, accumulated_grads will be stored in a simpler one
    state.setdefault('m', {})  # first moment vars
    state.setdefault('v', {})  # second moment vars
    state.setdefault('t', 0)   # timestamp
    state['t'] += 1
    for k in ['learning_rate', 'beta1', 'beta2', 'epsilon']:
        assert k in config, config.keys()
    
    var_index = 0 
    lr_t = config['learning_rate'] * np.sqrt(1 - config['beta2']**state['t']) / (1 - config['beta1']**state['t'])
    for current_layer_vars, current_layer_grads in zip(variables, gradients): 
        for current_var, current_grad in zip(current_layer_vars, current_layer_grads):
            var_first_moment = state['m'].setdefault(var_index, np.zeros_like(current_grad))
            var_second_moment = state['v'].setdefault(var_index, np.zeros_like(current_grad))
            
            # <YOUR CODE> #######################################
            # update `current_var_first_moment`, `var_second_moment` and `current_var` values
            #np.add(... , out=var_first_moment)
            #np.add(... , out=var_second_moment)
            #current_var -= ...
            
            np.add(config['beta1'] * var_first_moment, (1. - config['beta1']) * current_grad, out=var_first_moment)
            np.add(config['beta2'] * var_second_moment, (1. - config['beta2']) * current_grad ** 2, out=var_second_moment)
            current_var -= lr_t * var_first_moment / (var_second_moment ** .5 + config['epsilon'])
            
            # small checks that you've updated the state; use np.add for rewriting np.arrays values
            assert var_first_moment is state['m'].get(var_index)
            assert var_second_moment is state['v'].get(var_index)
            var_index += 1

In [ ]:
import scipy as sp
import scipy.signal
import skimage

class Conv2d(Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(Conv2d, self).__init__()
        assert kernel_size % 2 == 1, kernel_size
       
        stdv = 1./np.sqrt(in_channels)
        self.W = np.random.uniform(-stdv, stdv, size = (out_channels, in_channels, kernel_size, kernel_size))
        self.b = np.random.uniform(-stdv, stdv, size=(out_channels,))
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        
        self.gradW = np.zeros_like(self.W)
        self.gradb = np.zeros_like(self.b)
        
    def updateOutput(self, input):
        pad_size = self.kernel_size // 2
        # YOUR CODE ##############################
        # 1. zero-pad the input array
        # 2. compute convolution using scipy.signal.correlate(... , mode='valid')
        # 3. add bias value
        
        # self.output = ...
        
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        pad_size = self.kernel_size // 2
        # YOUR CODE ##############################
        # 1. zero-pad the gradOutput
        # 2. compute 'self.gradInput' value using scipy.signal.correlate(... , mode='valid')
        
        # self.gradInput = ...
        
        return self.gradInput
    
    def accGradParameters(self, input, gradOutput):
        pad_size = self.kernel_size // 2
        # YOUR CODE #############
        # 1. zero-pad the input
        # 2. compute 'self.gradW' using scipy.signal.correlate(... , mode='valid')
        # 3. compute 'self.gradb' - formulas like in Linear of ChannelwiseScaling layers
        
        # self.gradW = ...
        # self.gradb = ...
        pass
    
    def zeroGradParameters(self):
        self.gradW.fill(0)
        self.gradb.fill(0)
        
    def getParameters(self):
        return [self.W, self.b]
    
    def getGradParameters(self):
        return [self.gradW, self.gradb]
    
    def __repr__(self):
        s = self.W.shape
        q = 'Conv2d %d -> %d' %(s[1],s[0])
        return q

In [ ]:
class MaxPool2d(Module):
    def __init__(self, kernel_size):
        super(MaxPool2d, self).__init__()
        self.kernel_size = kernel_size
        self.gradInput = None
                    
    def updateOutput(self, input):
        input_h, input_w = input.shape[-2:]
        # your may remove these asserts and implement MaxPool2d with padding
        assert input_h % self.kernel_size == 0  
        assert input_w % self.kernel_size == 0
        
        # YOUR CODE #############################
        # self.output = ...
        # self.max_indices = ...
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        # YOUR CODE #############################
        # self.gradInput = ...
        return self.gradInput
    
    def __repr__(self):
        q = 'MaxPool2d, kern %d, stride %d' %(self.kernel_size, self.kernel_size)
        return q

In [ ]:
class Flatten(Module):
    def __init__(self):
         super(Flatten, self).__init__()
    
    def updateOutput(self, input):
        self.output = input.reshape(len(input), -1)
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        self.gradInput = gradOutput.reshape(input.shape)
        return self.gradInput
    
    def __repr__(self):
        return "Flatten"

### Test 0: Initialization (0.01 points)

In [ ]:
# do not change this cell
import sys

import numpy
import numpy as np
import unittest

import collections
import pickle
import io


numpy.variance = numpy.var


### Test 1: Linear layer (0.04 points)

In [ ]:
batch_size, n_in, n_out = 2, 3, 4
for d in data['test_Linear']:
    # layers initialization
    custom_layer = Linear(n_in, n_out)
    custom_layer.W = d['init_w']
    custom_layer.b = d['init_w_b']

    layer_input = d['input']
    next_layer_grad = d['next_layer_grad']

    # 1. check layer output
    custom_layer_output = custom_layer.updateOutput(layer_input)
    gt_output = d['output']
    assert np.allclose(gt_output, custom_layer_output, atol=1e-6)

    # 2. check layer input grad
    custom_layer_grad = custom_layer.updateGradInput(layer_input, next_layer_grad)
    gt_layer_grad = d['grad_output']
    assert np.allclose(gt_layer_grad, custom_layer_grad, atol=1e-6)

    # 3. check layer parameters grad
    custom_layer.accGradParameters(layer_input, next_layer_grad)
    weight_grad = custom_layer.gradW
    bias_grad = custom_layer.gradb
    torch_weight_grad = d['w_grad']
    torch_bias_grad = d['b_grad']
    assert np.allclose(torch_weight_grad, weight_grad, atol=1e-6)
    assert np.allclose(torch_bias_grad, bias_grad, atol=1e-6)

### Test 2: Softmax (0.1 points)

In [ ]:
np.random.seed(42)

batch_size, n_in = 2, 4
for d in data['test_SoftMax']:
    # layers initialization
    custom_layer = SoftMax()

    layer_input = d['input']
    next_layer_grad = d['next_layer_grad']

    # 1. check layer output
    custom_layer_output = custom_layer.updateOutput(layer_input)
    gt_output = d['output']
    assert np.allclose(gt_output, custom_layer_output, atol=1e-5)

    # 2. check layer input grad
    custom_layer_grad = custom_layer.updateGradInput(layer_input, next_layer_grad)
    assert np.allclose(d['grad_output'], custom_layer_grad, atol=1e-5)

### Test 3: LogSoftMax (0.1 points)

In [ ]:
np.random.seed(42)

batch_size, n_in = 2, 4
for d in data['test_LogSoftMax']:
    # layers initialization
    custom_layer = LogSoftMax()

    layer_input = d['input']
    next_layer_grad = d['next_layer_grad']

    # 1. check layer output
    custom_layer_output = custom_layer.updateOutput(layer_input)
    gt_output = d['output']
    np.allclose(gt_output, custom_layer_output, atol=1e-6)

    # 2. check layer input grad
    custom_layer_grad = custom_layer.updateGradInput(layer_input, next_layer_grad)
    gt_grad = d['grad_output']
    np.allclose(gt_grad, custom_layer_grad, atol=1e-6)

### Test 4: BatchNormalization (0.1 points)

In [ ]:
np.random.seed(42)

batch_size, n_in = 32, 16
for d in data['test_BatchNormalization']:
    # layers initialization
    alpha = 0.9
    custom_layer = BatchNormalization(alpha)
    custom_layer.train()
    init_moving_mean = d['init_moving_mean']
    init_moving_variance = d['init_moving_variance']
    custom_layer.moving_mean = init_moving_mean
    custom_layer.moving_variance = init_moving_variance

    layer_input = d['input']
    next_layer_grad = d['next_layer_grad']

    # 1. check layer output
    custom_layer_output = custom_layer.updateOutput(layer_input)
    gt_output = d['output']
    np.allclose(gt_output, custom_layer_output, atol=1e-6)

    # 2. check layer input grad
    custom_layer_grad = custom_layer.updateGradInput(layer_input, next_layer_grad)
    gt_grad = d['grad_output']
    # please, don't increase `atol` parameter, it's garanteed that you can implement batch norm layer
    # with tolerance 1e-5
    np.allclose(gt_grad, custom_layer_grad, atol=1e-5)

    # 3. check moving mean
    gt_running_mean = d['gt_moving_mean']
    gt_running_var = d['gt_moving_var']
    np.allclose(custom_layer.moving_mean, gt_running_mean)
    # we don't check moving_variance because pytorch uses slightly different formula for it:
    # it computes moving average for unbiased variance (i.e var*N/(N-1))
    #self.assertTrue(np.allclose(custom_layer.moving_variance, torch_layer.running_var.numpy()))

    # 4. check evaluation mode
    custom_layer.moving_variance = gt_running_var
    custom_layer.evaluate()
    custom_layer_output = custom_layer.updateOutput(layer_input)
    eval_output = d['eval_output']
    np.allclose(eval_output, custom_layer_output, atol=1e-6)

### Test 5: Sequential (0.1 points)

In [ ]:
np.random.seed(42)

batch_size, n_in = 2, 4
for d in data['test_Sequential']:
    # layers initialization
    alpha = 0.9
    custom_layer = Sequential()
    bn_layer = BatchNormalization(alpha)
    bn_layer.moving_mean = d['init_moving_mean']
    bn_layer.moving_variance = d['init_moving_var']

    custom_layer.add(bn_layer)
    scaling_layer = ChannelwiseScaling(n_in)
    scaling_layer.gamma = d['init_gamma']
    scaling_layer.beta = d['init_beta']
    custom_layer.add(scaling_layer)
    custom_layer.train()

    layer_input = d['input']
    next_layer_grad = d['next_layer_grad']

    # 1. check layer output
    custom_layer_output = custom_layer.updateOutput(layer_input)
    assert np.allclose(d['gt_output'], custom_layer_output, atol=1e-5)

    # 2. check layer input grad
    custom_layer_grad = custom_layer.backward(layer_input, next_layer_grad)
    assert np.allclose(d['gt_grad'], custom_layer_grad, atol=1e-5)

    # 3. check layer parameters grad
    weight_grad, bias_grad = custom_layer.getGradParameters()[1]
    torch_weight_grad = d['gt_weight_grad']
    torch_bias_grad = d['gt_bias_grad']
    assert np.allclose(torch_weight_grad, weight_grad, atol=1e-5)
    assert np.allclose(torch_bias_grad, bias_grad, atol=1e-5)

### Test 6: Dropout (0.075 points)

In [ ]:
np.random.seed(42)

batch_size, n_in = 2, 4
for _ in range(100):
    # layers initialization
    p = np.random.uniform(0.3, 0.7)
    layer = Dropout(p)
    layer.train()

    layer_input = np.random.uniform(-5, 5, (batch_size, n_in)).astype(np.float32)
    next_layer_grad = np.random.uniform(-5, 5, (batch_size, n_in)).astype(np.float32)

    # 1. check layer output
    layer_output = layer.updateOutput(layer_input)
    assert np.all(np.logical_or(np.isclose(layer_output, 0),
                                np.isclose(layer_output*(1.-p), layer_input)))

    # 2. check layer input grad
    layer_grad = layer.updateGradInput(layer_input, next_layer_grad)
    assert np.all(np.logical_or(np.isclose(layer_grad, 0),
                                np.isclose(layer_grad*(1.-p), next_layer_grad)))

    # 3. check evaluation mode
    layer.evaluate()
    layer_output = layer.updateOutput(layer_input)
    assert np.allclose(layer_output, layer_input)

    # 4. check mask
    p = 0.0
    layer = Dropout(p)
    layer.train()
    layer_output = layer.updateOutput(layer_input)
    assert np.allclose(layer_output, layer_input)

    p = 0.5
    layer = Dropout(p)
    layer.train()
    layer_input = np.random.uniform(5, 10, (batch_size, n_in)).astype(np.float32)
    next_layer_grad = np.random.uniform(5, 10, (batch_size, n_in)).astype(np.float32)
    layer_output = layer.updateOutput(layer_input)
    zeroed_elem_mask = np.isclose(layer_output, 0)
    layer_grad = layer.updateGradInput(layer_input, next_layer_grad)
    assert np.all(zeroed_elem_mask == np.isclose(layer_grad, 0))

    # 5. dropout mask should be generated independently for every input matrix element, not for row/column
    batch_size, n_in = 1000, 1
    p = 0.8
    layer = Dropout(p)
    layer.train()

    layer_input = np.random.uniform(5, 10, (batch_size, n_in)).astype(np.float32)
    layer_output = layer.updateOutput(layer_input)
    assert np.sum(np.isclose(layer_output, 0)) != layer_input.size

    layer_input = layer_input.T
    layer_output = layer.updateOutput(layer_input)
    assert np.sum(np.isclose(layer_output, 0)) != layer_input.size

### Test 7: LeakyReLU (0.05 points)

In [ ]:
np.random.seed(42)

batch_size, n_in = 2, 4
for d in data['test_LeakyReLU']:
    # layers initialization
    slope = d['slope']
    custom_layer = LeakyReLU(slope)

    layer_input = d['layer_input']
    next_layer_grad = d['next_layer_grad']

    # 1. check layer output
    custom_layer_output = custom_layer.updateOutput(layer_input)
    assert np.allclose(d['gt_output'], custom_layer_output, atol=1e-6)

    # 2. check layer input grad
    custom_layer_grad = custom_layer.updateGradInput(layer_input, next_layer_grad)
    assert np.allclose(d['gt_grad'], custom_layer_grad, atol=1e-6)

### Test 8: ELU (0.075 points)

In [ ]:
np.random.seed(42)

batch_size, n_in = 2, 4
for d in data['test_ELU']:
    # layers initialization
    alpha = 1.0
    custom_layer = ELU(alpha)

    layer_input = d['layer_input']
    next_layer_grad = d['next_layer_grad']

    # 1. check layer output
    custom_layer_output = custom_layer.updateOutput(layer_input)
    assert np.allclose(d['gt_output'], custom_layer_output, atol=1e-6)

    # 2. check layer input grad
    custom_layer_grad = custom_layer.updateGradInput(layer_input, next_layer_grad)
    assert np.allclose(d['gt_grad'], custom_layer_grad, atol=1e-6)

### Test 9: SoftPlus (0.1 points)

In [ ]:
np.random.seed(42)

batch_size, n_in = 2, 4
for d in data['test_SoftPlus']:
    # layers initialization
    custom_layer = SoftPlus()

    layer_input = d['layer_input']
    next_layer_grad = d['next_layer_grad']

    # 1. check layer output
    custom_layer_output = custom_layer.updateOutput(layer_input)
    assert np.allclose(d['gt_output'], custom_layer_output, atol=1e-6)

    # 2. check layer input grad
    custom_layer_grad = custom_layer.updateGradInput(layer_input, next_layer_grad)
    assert np.allclose(d['gt_grad'], custom_layer_grad, atol=1e-6)


### Test 10: ClassNLLCriterionUnstable (0.1 points)

In [ ]:
np.random.seed(42)

batch_size, n_in = 2, 4
for d in data['test_ClassNLLCriterionUnstable']:
    # layers initialization
    custom_layer = ClassNLLCriterionUnstable()

    layer_input = d['layer_input']
    target = d['target']

    # 1. check layer output
    custom_layer_output = custom_layer.updateOutput(layer_input, target)
    assert np.allclose(d['gt_output'], custom_layer_output, atol=1e-6)

    # 2. check layer input grad
    custom_layer_grad = custom_layer.updateGradInput(layer_input, target)
    assert np.allclose(d['gt_grad'], custom_layer_grad, atol=1e-6)

### Test 11: ClassNLLCriterion (0.05 points)

In [ ]:
np.random.seed(42)

batch_size, n_in = 2, 4
for d in data['test_ClassNLLCriterion']:
    # layers initialization
    custom_layer = ClassNLLCriterion()

    layer_input = d['layer_input']
    target = d['target']

    # 1. check layer output
    custom_layer_output = custom_layer.updateOutput(layer_input, target)
    assert np.allclose(d['gt_output'], custom_layer_output, atol=1e-6)

    # 2. check layer input grad
    custom_layer_grad = custom_layer.updateGradInput(layer_input, target)
    assert np.allclose(d['gt_grad'], custom_layer_grad, atol=1e-6)

### Test 12: Adam (0.1 points)

In [ ]:
state = {}
config = {'learning_rate': 1e-3, 'beta1': 0.9, 'beta2':0.999, 'epsilon':1e-8}
variables = [[np.arange(10).astype(np.float64)]]
gradients = [[np.arange(10).astype(np.float64)]]
adam_optimizer(variables, gradients, config, state)
assert np.allclose(state['m'][0], np.array([0. , 0.1, 0.2, 0.3, 0.4, 0.5,
                                                     0.6, 0.7, 0.8, 0.9]))
assert np.allclose(state['v'][0], np.array([0., 0.001, 0.004, 0.009, 0.016, 0.025,
                                                     0.036, 0.049, 0.064, 0.081]))
assert state['t'] == 1
assert np.allclose(variables[0][0], np.array([0., 0.999, 1.999, 2.999, 3.999, 4.999,
                                                       5.999, 6.999, 7.999, 8.999]))
adam_optimizer(variables, gradients, config, state)
assert np.allclose(state['m'][0], np.array([0., 0.19, 0.38, 0.57, 0.76, 0.95, 1.14,
                                                     1.33, 1.52, 1.71]))
assert np.allclose(state['v'][0], np.array([0., 0.001999, 0.007996, 0.017991,
                                                     0.031984, 0.049975, 0.071964, 0.097951,
                                                     0.127936, 0.161919]))
assert state['t'] == 2
assert np.allclose(variables[0][0], np.array([0., 0.998, 1.998, 2.998, 3.998, 4.998,
                                                       5.998, 6.998, 7.998, 8.998]))

### Test 13: Conv2d (0.35 points)

In [ ]:
np.random.seed(42)

batch_size, n_in, n_out = 2, 3, 4
h,w = 5,6
kern_size = 3
for d in data['test_Conv2d']:
    # layers initialization
    custom_layer = Conv2d(n_in, n_out, kern_size)
    custom_layer.W = d['init_w']
    custom_layer.b = d['init_b']

    layer_input = d['input']
    next_layer_grad = d['next_layer_grad']

    # 1. check layer output
    custom_layer_output = custom_layer.updateOutput(layer_input)
    assert np.allclose(d['gt_output'], custom_layer_output, atol=1e-6)

    # 2. check layer input grad
    custom_layer_grad = custom_layer.updateGradInput(layer_input, next_layer_grad)
    assert np.allclose(d['gt_grad'], custom_layer_grad, atol=1e-6)

    # 3. check layer parameters grad
    custom_layer.accGradParameters(layer_input, next_layer_grad)
    weight_grad = custom_layer.gradW
    bias_grad = custom_layer.gradb
    assert np.allclose(d['gt_weight_grad'], weight_grad, atol=1e-6, )
    assert np.allclose(d['gt_bias_grad'], bias_grad, atol=1e-6)


### Test 13: MaxPool2d (0.15 points)

In [ ]:
np.random.seed(42)

batch_size, n_in = 2, 3
h,w = 4,6
kern_size = 2
for d in data['test_MaxPool2d']:
    # layers initialization
    custom_layer = MaxPool2d(kern_size)

    layer_input = d['input']
    next_layer_grad = d['next_layer_grad']

    # 1. check layer output
    custom_layer_output = custom_layer.updateOutput(layer_input)
    assert np.allclose(d['gt_output'], custom_layer_output, atol=1e-6)

    # 2. check layer input grad
    custom_layer_grad = custom_layer.updateGradInput(layer_input, next_layer_grad)
    assert np.allclose(d['gt_grad'], custom_layer_grad, atol=1e-6)